## Importar librerías

In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Librerias

In [0]:
def crear_boxplots_seaborn_df(dataframe, col1, col2, col3):
    """
    Crea un gráfico de boxplot con tres subplots paralelos para las columnas de un DataFrame usando seaborn.
    """
    # Crear una figura y tres subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle('Boxplots de Tres Variables', fontsize=16)

    # Crear los boxplots con seaborn
    sns.boxplot(y=dataframe[col1], ax=axes[0])
    axes[0].set_title(col1)
    axes[0].set_ylabel('Valores')

    sns.boxplot(y=dataframe[col2], ax=axes[1])
    axes[1].set_title(col2)

    sns.boxplot(y=dataframe[col3], ax=axes[2])
    axes[2].set_title(col3)

    # Ajustar el diseño
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [0]:
def cap_outliers_at_75th_percentile(dataframe, variables):
    """
    Capea los valores de las columnas especificadas en el DataFrame al percentil 75.
    Para cada variable, todos los valores mayores al percentil 75 serán reemplazados por el valor del percentil.
    """
    df = dataframe.copy()  # Crear una copia para no modificar el original
    for var in variables:
        if var in df.columns:
            p75 = df[var].quantile(0.95)  # Calcular el percentil 75
            df[var] = df[var].apply(lambda x: min(x, p75))  # Imputar el valor del percentil 75
        else:
            print(f"Advertencia: {var} no se encuentra en el DataFrame.")
    return df

## Leer bases

In [0]:
ruta_base = "workspace.default.base"

In [0]:
# Leer la tabla usando Spark
df_sp = spark.table(ruta_base)

In [0]:
df = df_sp.toPandas()


## Procesamiento

In [0]:
df = df.fillna(0)
df = df.replace([np.inf, -np.inf], 0)

In [0]:
df.shape

In [0]:
df["flg_churn"].value_counts(1)

In [0]:
df['tipo_producto'].value_counts()

In [0]:
df = pd.get_dummies(df, columns=['tipo_producto'], prefix=None)

In [0]:
df["tipo_producto_producto_a"] = df["tipo_producto_producto_a"].astype(int)
df["tipo_producto_producto_b"] = df["tipo_producto_producto_b"].astype(int)
df["tipo_producto_producto_c"] = df["tipo_producto_producto_c"].astype(int)

In [0]:
df['segmento_cliente'].value_counts()

In [0]:
df = pd.get_dummies(df, columns=['segmento_cliente'], prefix='segmento')

In [0]:
df['segmento_1'] = df['segmento_1'].astype(int)
df['segmento_2'] = df['segmento_2'].astype(int)
df['segmento_3'] = df['segmento_3'].astype(int)
df['segmento_4'] = df['segmento_4'].astype(int)
df['segmento_5'] = df['segmento_5'].astype(int)
df['segmento_6'] = df['segmento_6'].astype(int)
df['segmento_7'] = df['segmento_7'].astype(int)
df['segmento_8'] = df['segmento_8'].astype(int)
df['segmento_9'] = df['segmento_9'].astype(int)
df['segmento_10'] = df['segmento_10'].astype(int)

## Tratamiento nulos

In [0]:
df.isnull().sum()[df.isnull().sum()>0]

## Tratamiento outliers

In [0]:
crear_boxplots_seaborn_df(df, 'monto_1m', 'cantidad_1m', 'incidencias_a')

In [0]:
variables_analizar_outliers = ['monto_1m', 'monto_2m', 'monto_3m', 'monto_4m',
       'monto_5m', 'monto_6m', 'cantidad_1m', 'cantidad_2m', 'cantidad_3m', 'cantidad_6m']

In [0]:
df = cap_outliers_at_75th_percentile(df, variables_analizar_outliers)

In [0]:
crear_boxplots_seaborn_df(df, 'monto_1m', 'cantidad_1m', 'incidencias_a')

## Segmento alto valor

In [0]:
df['monto_1m'].describe()

In [0]:
df['cantidad_1m'].describe()

In [0]:
df = df.drop(columns = ['__index_level_0__', '__index_level_0___x', '__index_level_0___y'])
df['segmento_0'] = df['segmento_0'].map(int)

In [0]:
cond_alto_valor = (df["monto_1m"]>8.5) & (df["cantidad_1m"]>3.5)

In [0]:
df_alto_valor = df[cond_alto_valor]

## Exportar bases

In [0]:
df.dtypes

In [0]:
# Convertir a Spark DataFrame
df_spark_export = spark.createDataFrame(df)

# Guardar en el catálogo
df_spark_export.write.mode("overwrite").saveAsTable("default.base_modelo")

In [0]:
# Convertir a Spark DataFrame
df_alto_valor_spark_export = spark.createDataFrame(df_alto_valor)

# Guardar en el catálogo
df_alto_valor_spark_export.write.mode("overwrite").saveAsTable("default.base_modelo_alto_valor")